In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
from IPython.display import HTML
from matplotlib import animation
from ipywidgets import interact, widgets

# Slightly incorrect implementation

In [ ]:
def test_advection(T=1,limit=True,order=3,nu=0.9,RKM='SSP33',IC='smooth'):

    v = 1

    def f(q):
        return v*q

    nghost = 2
    m = 50      # number of cells
    dx = 1./m   # Size of 1 grid cell
    x = np.linspace(-(2*nghost-1)*dx/2,1.+(2*nghost-1)*dx/2,m+2*nghost)
    t = 0.      # Initial time
    dt = nu * dx  # Time step

    if IC=='smooth':
        Q = np.exp(-200*(x-0.2)**2)    # Initial data
    elif IC=='square':
        Q = 1.*(x<0.2)*(x>0.1)
    H_iph = np.zeros_like(x)
    ql = np.zeros_like(x)
    qr = np.zeros_like(x)
    ubar_L_iph = np.zeros_like(x)
    H_L_iph = np.zeros_like(x)
    H_iph = np.zeros_like(x)
    f_iph = np.zeros_like(x)
    fstar_iph = np.zeros_like(x)
    umax = np.zeros_like(x)
    umin = np.zeros_like(x)

    rkm = rk.loadRKM(RKM)
    rkm = rkm.__num__()

    t = 0. # current time
    QQ = [Q.copy()] # values at each time step
    tt = np.zeros(1) # time points for ww
    tt[0] = t
    b = rkm.b
    s = len(rkm)
    y = np.zeros((s, np.size(Q))) # stage values
    G = np.zeros((s, np.size(Q))) # stage derivatives

    def donor(q):
        return q, q

    def pw_poly_recon(q,order=3):
        if order==1:
            ql[1:-1] = q[1:-1]
            qr[1:-1] = q[1:-1]
        if order==3:
            ql[1:-1] = (2.*q[:-2] + 5.*q[1:-1] - q[2:])/6.
            qr[1:-1] = (-q[:-2] + 5.*q[1:-1] + 2.*q[2:])/6.
        elif order==5:
            ql[2:-2] = (-5.*q[:-4] + 60.*q[1:-3] + 90.*q[2:-2] - 20.*q[3:-1] + 3.*q[4:])/128.
            qr[2:-2] =  (3.*q[:-4] - 20.*q[1:-3] + 90.*q[2:-2] + 60.*q[3:-1] - 5.*q[4:])/128.
        return ql, qr

    d = np.abs(v)/2.

    while t < T and not np.isclose(t, T):
        if t + dt > T:
            nu = nu*(T-t)/dt
            dt = T - t

        for i in range(s):
            y[i,:] = Q.copy()
            for j in range(i):
                y[i,:] += rkm.A[i,j]*G[j,:]
            # Reconstruction:
            ql, qr = pw_poly_recon(y[i,:],order=order)

            ubar_L_iph[1:-1] = 0.5*(Q[2:]   + Q[1:-1]) - 0.25*(f(Q[2:])  -f(Q[1:-1]))/d
            H_L_iph[:-1] = 0.5*(f(Q[:-1])+f(Q[1:])) - d*(Q[1:]-Q[:-1])
            H_iph[:-1] = 0.5*(f(qr[:-1])+f(ql[1:])) - d*(ql[1:]-qr[:-1])
            f_iph = H_L_iph - H_iph

            umax[1:-1] = np.maximum(Q[:-2],np.maximum(Q[1:-1],Q[2:]))
            umin[1:-1] = np.minimum(Q[:-2],np.minimum(Q[1:-1],Q[2:]))
            fstar_iph[:-1] = (f_iph[:-1]>0)*np.minimum(f_iph[:-1], 
                                                       2*d*np.minimum(umax[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umin[1:]))\
                            +(f_iph[:-1]<0)*np.maximum(f_iph[:-1],
                                                       2*d*np.maximum(umin[:-1]-ubar_L_iph[:-1],ubar_L_iph[:-1]-umax[1:]))
        
            # Periodic boundaries:
            H_L_iph[:nghost] = H_L_iph[-2*nghost:-nghost]
            H_L_iph[-nghost:] = H_L_iph[nghost:2*nghost]
            fstar_iph[:nghost] = fstar_iph[-2*nghost:-nghost]
            fstar_iph[-nghost:] = fstar_iph[nghost:2*nghost]
            # Next two lines only needed for testing
            f_iph[:nghost] = f_iph[-2*nghost:-nghost]
            f_iph[-nghost:] = f_iph[nghost:2*nghost]
            if limit:
                G[i,1:-1] = - nu*(H_L_iph[1:-1]-H_L_iph[:-2]) + nu*(fstar_iph[1:-1]-fstar_iph[:-2])
            else:
                G[i,1:-1] = - nu*(H_L_iph[1:-1]-H_L_iph[:-2]) + nu*(f_iph[1:-1]-f_iph[:-2])

        Q = Q + sum([b[i]*G[i,:] for i in range(s)])
        Q[:nghost] = Q[-2*nghost:-nghost]
        Q[-nghost:] = Q[nghost:2*nghost]
        t += dt
        tt = np.append(tt, t)
        QQ.append(Q.copy())

    plt.plot(x[nghost:-nghost],Q[nghost:-nghost],linewidth = 2)
    plt.title('t = '+str(t));
    plt.plot(x[nghost:-nghost],QQ[0][nghost:-nghost]);
    print(np.min(Q[nghost:-nghost]))
    print(np.argmin(Q[nghost:-nghost]))

In [ ]:
interact(test_advection,T=widgets.IntSlider(min=0,max=10,step=1,value=1),
            order=widgets.IntSlider(min=1,max=5,step=2),
            nu=widgets.FloatSlider(min=0.1,max=1.,step=0.1,value=0.5),
            RKM=widgets.Dropdown(options=['FE', 'SSP33', 'SSP104','BS5'],value='FE',description='RK method:'),
            IC=widgets.Dropdown(options=['smooth', 'square'],value='smooth',description='Initial cond.:'),
        
        );